In [1]:
import sys
sys.path.append(r"C:\Users\Alex\Desktop\New folder\test\mido-master")
sys.path.append(r"C:\Users\Alex\Desktop\New folder\test\pretty-midi-master")

import mido
import pretty_midi
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
files = []
i=0
#Grabs files 
for entry in os.scandir('Midi_file'):
    if entry.is_file():
        if i < 15:
            files.append(entry.path)
            i=i+1

In [22]:
duration = []
end_time = []
#determines timestamp
for midi in files:
    midi_data = pretty_midi.PrettyMIDI(midi)
    for instrument in midi_data.instruments:
        for i,note in enumerate(instrument.notes):
                duration.append(note.end - note.start)
        end_time.append(note.end)
        break
duration.sort()
bucket_length = int(len(duration)/4)
duration_bucket_1 = bucket_length
duration_bucket_2 = bucket_length * 2
duration_bucket_3 = bucket_length * 3
step = [0,0,0,0,0]
step[1] = duration[duration_bucket_1]
step[2] = duration[duration_bucket_2]
step[3] = duration[duration_bucket_3]
step[4] = duration[(len(duration) - 100)]
print(step[1])
print(step[2])
print(step[3])
print(step[4])

0.09519574999998781
0.21626300000000143
0.3671193750000157
2.257056000000034


In [4]:
import numpy as np
import tensorflow as tf
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [5]:
song = []
#grabs notes from songs
for midi in files:
    current_step = 0.0
    current_index = 0
    midi_data = pretty_midi.PrettyMIDI(midi)

    note = []
    for instrument in midi_data.instruments:
        if(instrument.name=="Drum kit 2"):
            continue
        if not instrument.is_drum:
            for n in instrument.notes:
                #determines duration lenght and timestamp bucket for each
                time = n.end - n.start
                if (time < duration[duration_bucket_1]):
                    time = 1
                elif (time < duration[duration_bucket_2]):
                    time = 2
                elif (time < duration[duration_bucket_3]):
                    time = 3
                else:
                    time = 4
                
                
                note.append(time)
                note.append(n.pitch)
                note.append(n.velocity)
                
                #converts note into one value string
                song.append('.'.join(str(i) for i in note))
                note.clear()
               
            break
#number of note types
n_vocab = len(set(song))
#collection of each note_type
pitchnames = sorted(set(item for item in song))   

#gives an int value to each note type
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

#main input vector
network_input = []
#output vector
network_output = []
#notes in sequence
sequence_length = 100

#sets up the list of 100 note sequences
for i in range(0, len(song) - sequence_length, 1):
    sequence_in = song[i:i + sequence_length]
    sequence_out = song[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
#how many total sequences    
n_patterns = len(network_input)

#converts to lstm readable matrix
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
#normalizes note values
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

print(network_input.shape)

(19045, 100, 1)


In [6]:
# Building the model architecture
#network_input.shape=(time, pitch, velocity)
model = Sequential()  
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))  
model.add(Dropout(0.2))                                                # prevents overfitting
model.add(LSTM(512, return_sequences=True))  
model.add(Flatten())  
model.add(Dense(256))  
model.add(Dropout(0.3))  
model.add(Dense(n_vocab))                                              # each input node becomes connected to output node
model.add(Activation('softmax'))  
model.compile(loss='categorical_crossentropy', optimizer='adam')    # optimizer= how it learns while loss= how well it learns

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               13107456  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5177)              1330489   
__________

In [7]:
from keras.callbacks import ModelCheckpoint
    
# Create checkpoint to save the best model weights.
filepath = filepath = r"C:\Users\Alex\Desktop\model_results\model--{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=[checkpoint])

Epoch 1/200
  256/19045 [..............................] - ETA: 18:33 - loss: 8.5598

KeyboardInterrupt: 

In [11]:
# ignore from here down until top works

# Building the model architecture
model = Sequential()  
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))  
model.add(Dropout(0.2))                                                # prevents overfitting
model.add(LSTM(512, return_sequences=True))  
model.add(Flatten())  
model.add(Dense(256))  
model.add(Dropout(0.3))  
model.add(Dense(n_vocab))                                              # each input node becomes connected to output node
model.add(Activation('softmax'))  
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')    # optimizer= how it learns while loss= how well it learns

model.load_weights(r"C:\Users\Alex\Desktop\model_results\model--64-0.1543-bigger.hdf5")
# Pick a random integer  
start = np.random.randint(0, len(network_input)-1)  
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))  
    # pick a random sequence from the input as a starting point for the prediction  

pattern = network_input[start]

prediction_output = []  
    #print('Generating notes........')  
    # generate 500 notes  
for note_index in range(500):  
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))  
    prediction_input = prediction_input / float(n_vocab)  
    prediction = model.predict(prediction_input, verbose=0)  
         # Predicted output is the argmax(P(h|D))  
    index = np.argmax(prediction)  
         # Mapping the predicted interger back to the corresponding note  
    result = int_to_note[index]  
         # Storing the predicted output  
    prediction_output.append(result) 
    pattern = np.append(pattern,index)  
         # Next input to the model  
    pattern = pattern[1:len(pattern)]  
print(prediction_output)







['2.43.71', '2.43.71', '2.52.63', '2.43.71', '2.46.71', '2.52.63', '2.46.71', '2.43.71', '2.52.63', '3.46.65', '3.52.65', '3.43.65', '4.60.35', '4.48.27', '1.44.25', '1.56.31', '4.53.28', '4.41.22', '4.44.27', '4.56.35', '2.60.30', '2.48.24', '4.53.27', '4.65.35', '4.56.28', '4.68.36', '2.60.25', '2.72.31', '4.65.31', '4.77.40', '4.53.26', '4.65.33', '4.72.35', '2.74.33', '4.64.26', '4.67.26', '1.74.33', '1.76.29', '1.74.33', '1.76.29', '1.74.33', '1.76.30', '1.74.33', '1.76.29', '1.74.33', '1.76.30', '1.74.28', '1.76.33', '1.74.28', '2.72.30', '2.76.35', '2.74.31', '4.71.26', '4.65.26', '4.67.22', '4.64.19', '4.72.35', '4.49.27', '4.61.35', '2.46.25', '2.58.31', '4.42.22', '4.54.28', '4.58.35', '4.46.27', '2.49.24', '2.61.30', '4.66.35', '4.54.27', '4.58.28', '4.70.36', '2.73.31', '2.61.25', '4.66.31', '4.78.40', '4.66.33', '4.54.26', '4.73.35', '2.75.33', '4.68.26', '4.65.26', '1.75.28', '1.77.25', '1.75.28', '1.77.25', '1.75.28', '1.77.25', '1.75.29', '1.77.25', '1.75.29', '1.77.26'

In [16]:
temp_song = []
temp_note = []
temp_value = 0
for converted_note in prediction_output:
    for char in converted_note:
        if char != '.':
            temp_value = temp_value * 10 + int(char) 
        if char == '.':
            temp_note.append(temp_value)
            temp_value = 0
    temp_note.append(temp_value)
    temp_value = 0
    temp_song.append(temp_note)
    temp_note = []
print(temp_song)            

[[2, 43, 71], [2, 43, 71], [2, 52, 63], [2, 43, 71], [2, 46, 71], [2, 52, 63], [2, 46, 71], [2, 43, 71], [2, 52, 63], [3, 46, 65], [3, 52, 65], [3, 43, 65], [4, 60, 35], [4, 48, 27], [1, 44, 25], [1, 56, 31], [4, 53, 28], [4, 41, 22], [4, 44, 27], [4, 56, 35], [2, 60, 30], [2, 48, 24], [4, 53, 27], [4, 65, 35], [4, 56, 28], [4, 68, 36], [2, 60, 25], [2, 72, 31], [4, 65, 31], [4, 77, 40], [4, 53, 26], [4, 65, 33], [4, 72, 35], [2, 74, 33], [4, 64, 26], [4, 67, 26], [1, 74, 33], [1, 76, 29], [1, 74, 33], [1, 76, 29], [1, 74, 33], [1, 76, 30], [1, 74, 33], [1, 76, 29], [1, 74, 33], [1, 76, 30], [1, 74, 28], [1, 76, 33], [1, 74, 28], [2, 72, 30], [2, 76, 35], [2, 74, 31], [4, 71, 26], [4, 65, 26], [4, 67, 22], [4, 64, 19], [4, 72, 35], [4, 49, 27], [4, 61, 35], [2, 46, 25], [2, 58, 31], [4, 42, 22], [4, 54, 28], [4, 58, 35], [4, 46, 27], [2, 49, 24], [2, 61, 30], [4, 66, 35], [4, 54, 27], [4, 58, 28], [4, 70, 36], [2, 73, 31], [2, 61, 25], [4, 66, 31], [4, 78, 40], [4, 66, 33], [4, 54, 26]

In [26]:
start_time = 0
output_song = pretty_midi.PrettyMIDI()
output_piano = pretty_midi.instrument_name_to_program('Piano')
piano = pretty_midi.Instrument(program=output_piano)
for item in temp_song:
    end_time = (step[item[0]] + start_time)
    converted_note = pretty_midi.Note(velocity=item[2],pitch=item[1],start=start_time,end=end_time)
    piano.notes.append(converted_note)
    start_time = end_time
output_song.instruments.append(piano)    
output_song.write('output_song.mid')

ValueError: Piano is not a valid General MIDI instrument name.

In [ ]:
def train_network():  
    epochs = 200 
    
    notes = get_notes()  
    print('Notes processed') 
    
    n_vocab = len(set(notes))  
    print('Vocab generated')
    
    # create input and output sequences
    network_in, network_out = prepare_sequences(notes, n_vocab)  
    print('Input and Output processed')  
    
    model = create_network(network_in, n_vocab)  
    print('Model created') 
    
    return model  

    print('Training in progress')  
    
    train(model, network_in, network_out, epochs)  
    print('Training completed')  


In [ ]:
# Train model
train_network()

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):  
    """ Generate notes from the neural network based on a sequence of notes """  
   # Pick a random integer  
    start = np.random.randint(0, len(network_input)-1)  
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))  
    # pick a random sequence from the input as a starting point for the prediction  
    pattern = network_input[start]  
    prediction_output = []  
    print('Generating notes........')  
    # generate 500 notes  
    for note_index in range(500):  
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))  
        prediction_input = prediction_input / float(n_vocab)  
        prediction = model.predict(prediction_input, verbose=0)  
         # Predicted output is the argmax(P(h|D))  
        index = np.argmax(prediction)  
         # Mapping the predicted interger back to the corresponding note  
        result = int_to_note[index]  
         # Storing the predicted output  
        prediction_output.append(result)  
        pattern.append(index)  
         # Next input to the model  
        pattern = pattern[1:len(pattern)]  
    print('Notes Generated...')  
    return prediction_output  


In [ ]:
 ### Converts the predicted output to midi format  
create_midi(prediction_output)  


In [ ]:
generate_notes(model, network_input, pitchnames, n_vocab)